In [8]:
import urllib.request

def gather_input_data(url, sessionId, transform=lambda x: str(x, "utf-8").strip('\n')):
    request = urllib.request.Request(url)
    request.add_header("cookie", "session={}".format(sessionId)) # Source the data directly from AoC

    values = []
    with urllib.request.urlopen(request) as data:
        for line in data:
            values.append(transform(line))

    return values

def get_data(day, year=2023):
    with open('sessionID') as f:
        sessionId = f.readlines()[0].strip()
    url = "https://adventofcode.com/%d/day/%d/input"%(year,day)
    data = gather_input_data(url, sessionId)
    return data

from collections import defaultdict

In [18]:
bricks = []
for line in get_data(22):
    start, stop = line.split('~')
    start = list(map(int, start.split(',')))
    stop = list(map(int, stop.split(',')))
    bricks.append((start, stop))
bricks.sort(key=lambda x: x[0][2])
n = len(bricks)

In [19]:
x = []
y = []
z =[]
for (s,e) in bricks:
    x.extend([s[0],e[0]])
    y.extend([s[1],e[1]])
    z.extend([s[2],e[2]])
print(max(x), max(y), max(z))
print(min(x), min(y), min(z))

9 9 362
0 0 1


In [20]:
highest = defaultdict(lambda:(0,-1))
bad = set()
graph = [[] for i in range(n)]
for idx,b in enumerate(bricks):
    mxh = -1
    support_set = set()
    for x in range(b[0][0], b[1][0]+1):
        for y in range(b[0][1], b[1][1]+1):
            if highest[x,y][0] + 1 > mxh:
                mxh = highest[x,y][0] + 1
                support_set = {highest[x,y][1]}
            elif highest[x,y][0] + 1 == mxh:
                support_set.add(highest[x,y][1])
    
    for x in support_set:
        if x != -1:
            graph[x].append(idx)

    if len(support_set) == 1:
        bad.add(support_set.pop())
    
    fall = b[0][2] - mxh
    if fall > 0:
        b[0][2] -= fall
        b[1][2] -= fall

    for x in range(b[0][0], b[1][0]+1):
        for y in range(b[0][1], b[1][1]+1):
            highest[x,y] = (b[1][2], idx)

print(len(bricks)-len(bad)-1) # -1 for "ground" brick

def count(idx, graph):
    indeg = [0 for __ in range(n)]
    for j in range(n):
        for i in graph[j]:
            indeg[i] += 1
    q = [idx]
    count = -1
    while len(q) > 0:
        count += 1
        x = q.pop()
        for i in graph[x]:
            indeg[i] -= 1
            if indeg[i] == 0:
                q.append(i)

    return count

print(sum(count(x, graph) for x in range(n)))

497
95059
